In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report

In [3]:
# Load data
# ======================
df = pd.read_csv("GYM.csv")

X = df[["Gender", "Goal", "BMI Category"]]
y = df[["Exercise Schedule", "Meal Plan"]]

In [4]:
df

,Gender,Goal,BMI Category,Exercise Schedule,Meal Plan
0,Female,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
1,Male,fat_burn,Underweight,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
2,Male,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
3,Male,muscle_gain,Overweight,"High-intensity interval training (HIIT), Cardi...","Low-carb, high-fiber diet: Avocado, grilled fi..."
4,Female,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
...,...,...,...,...,...
79995,Male,fat_burn,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
79996,Female,fat_burn,Underweight,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
79997,Female,muscle_gain,Obesity,"Low-impact cardio, Swimming, and 10000 steps w...","Low-calorie, nutrient-dense diet with portion ..."
79998,Male,fat_burn,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...


In [5]:
X

,Gender,Goal,BMI Category
0,Female,muscle_gain,Normal weight
1,Male,fat_burn,Underweight
2,Male,muscle_gain,Normal weight
3,Male,muscle_gain,Overweight
4,Female,muscle_gain,Normal weight
...,...,...,...
79995,Male,fat_burn,Normal weight
79996,Female,fat_burn,Underweight
79997,Female,muscle_gain,Obesity
79998,Male,fat_burn,Normal weight


In [6]:
y

,Exercise Schedule,Meal Plan
0,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
1,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
2,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
3,"High-intensity interval training (HIIT), Cardi...","Low-carb, high-fiber diet: Avocado, grilled fi..."
4,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
...,...,...
79995,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
79996,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
79997,"Low-impact cardio, Swimming, and 10000 steps w...","Low-calorie, nutrient-dense diet with portion ..."
79998,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...


In [7]:
# Train / test split FIRST (no leakage)
# ======================
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
# Encode labels
# ======================
exercise_encoder = LabelEncoder()
meal_encoder = LabelEncoder()

y_ex_train = exercise_encoder.fit_transform(y_train_raw["Exercise Schedule"])
y_ex_test  = exercise_encoder.transform(y_test_raw["Exercise Schedule"])

y_me_train = meal_encoder.fit_transform(y_train_raw["Meal Plan"])
y_me_test  = meal_encoder.transform(y_test_raw["Meal Plan"])

In [9]:
# Encode features
# ======================
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

X_train = ohe.fit_transform(X_train_raw)
X_test  = ohe.transform(X_test_raw)

In [10]:
# Convert to TensorFlow tensors
# ======================
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test  = tf.convert_to_tensor(X_test, dtype=tf.float32)

y_ex_train = tf.convert_to_tensor(y_ex_train, dtype=tf.int32)
y_ex_test  = tf.convert_to_tensor(y_ex_test, dtype=tf.int32)

y_me_train = tf.convert_to_tensor(y_me_train, dtype=tf.int32)
y_me_test  = tf.convert_to_tensor(y_me_test, dtype=tf.int32)

In [11]:
# Model definition (shared trunk + two heads)
# ======================
input_dim = X_train.shape[1]
n_exercise = len(np.unique(y_ex_train.numpy()))
n_meal = len(np.unique(y_me_train.numpy()))

inputs = keras.Input(shape=(input_dim,))

x = layers.Dense(128, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)

exercise_output = layers.Dense(
    n_exercise, activation=None, name="exercise"
)(x)

meal_output = layers.Dense(
    n_meal, activation=None, name="meal"
)(x)

model = keras.Model(
    inputs=inputs,
    outputs=[exercise_output, meal_output]
)

In [ ]:
# Compile
# ======================
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss={
        "exercise": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        "meal": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    },
    loss_weights={
        "exercise": 1.0,
        "meal": 2.0,   # loss_ex + 2 * loss_me
    }
)

In [13]:
# Training
# ======================
model.fit(
    X_train,
    {
        "exercise": y_ex_train,
        "meal": y_me_train
    },
    epochs=50,
    batch_size=None,  # full-batch
    verbose=2
)

Epoch 1/50
2000/2000 - 3s - 1ms/step - exercise_loss: 0.0038 - loss: 0.0096 - meal_loss: 0.0029
Epoch 2/50
2000/2000 - 2s - 935us/step - exercise_loss: 3.5539e-09 - loss: 3.5539e-09 - meal_loss: 0.0000e+00
Epoch 3/50
2000/2000 - 2s - 934us/step - exercise_loss: 2.9057e-10 - loss: 2.9057e-10 - meal_loss: 0.0000e+00
Epoch 4/50
2000/2000 - 2s - 901us/step - exercise_loss: 0.0000e+00 - loss: 0.0000e+00 - meal_loss: 0.0000e+00
Epoch 5/50
2000/2000 - 2s - 911us/step - exercise_loss: 0.0000e+00 - loss: 0.0000e+00 - meal_loss: 0.0000e+00
Epoch 6/50
2000/2000 - 2s - 902us/step - exercise_loss: 0.0000e+00 - loss: 0.0000e+00 - meal_loss: 0.0000e+00
Epoch 7/50
2000/2000 - 2s - 874us/step - exercise_loss: 0.0000e+00 - loss: 0.0000e+00 - meal_loss: 0.0000e+00
Epoch 8/50
2000/2000 - 2s - 880us/step - exercise_loss: 0.0000e+00 - loss: 0.0000e+00 - meal_loss: 0.0000e+00
Epoch 9/50
2000/2000 - 2s - 931us/step - exercise_loss: 0.0000e+00 - loss: 0.0000e+00 - meal_loss: 0.0000e+00
Epoch 10/50
2000/2000 - 

In [14]:
# Evaluation
# ======================
ex_logits, meal_logits = model.predict(X_test)

ex_preds = np.argmax(ex_logits, axis=1)
meal_preds = np.argmax(meal_logits, axis=1)

print("Exercise Schedule:")
print(classification_report(y_ex_test.numpy(), ex_preds))

print("Meal Plan:")
print(classification_report(y_me_test.numpy(), meal_preds))


500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step
Exercise Schedule:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3993
           1       1.00      1.00      1.00      4219
           2       1.00      1.00      1.00      3838
           3       1.00      1.00      1.00      3950

    accuracy                           1.00     16000
   macro avg       1.00      1.00      1.00     16000
weighted avg       1.00      1.00      1.00     16000

Meal Plan:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3950
           1       1.00      1.00      1.00      4219
           2       1.00      1.00      1.00      3838
           3       1.00      1.00      1.00      3993

    accuracy                           1.00     16000
   macro avg       1.00      1.00      1.00     16000
weighted avg       1.00      1.00      1.00     16000

